# Lab 3: Functional programming, and declarative patterns


__Student:__ marja987

__Student:__ abcde123

Disclaimer: Functional programming in Python does not always lead to the fastest possible code, and is often not considered the *pythonic* approach. However, functional programming is the basis for many concurrent systems (the MapReduce programming model which many big data systems, e.g. Hadoop, relies on gets its name from the *map* and *reduce* functions mentioned below). Python is a multi-paradigm language, and functional programming is one of the main paradigms one can use. To understand how and when to do this, it is necessary to do things in a non-*pythonic* way in order to cover the basics.

## General instructions

In this lab there are some general rules you should keep in mind to make sure you are on the correct path in your solutions.

#### Rules
1. You are not allowed to use `while` or `for` statements unless this is explicitly allowed in the task.
2. You are not allowed to use global variables (other than for functions defined in the global environment).
3. Code stubs should be viewed as fixed, you are only allowed to add code, the only code you are allowed to change is `pass` statements, which you should remove.
4. You should refrain from using the `list` datatype unless otherwise specified and instead use `tuple`. One of the strengths of functional programming is its focus on immutable data types (this is why functional programming and concurrency goes so well together). Incidentally, one might find speedups when using the immutable tuples instead of lists.

#### Advice
1. Avoid local variables unless you are certain they are necessary, in most cases you won't need to use local variables. (altermatively, use local variables to your hearts content, but when your solution works, try to eliminate them, you should be able to eliminate most of them, over time, you might find that you don't need them.)

## 2 Higher order functions (HOF)
 
A _higher-order function_ is a function which operates on other functions. What this means exactly is disputed, but we will call any function which returns a function or takes a function as an argument a higher-order function. (Conversely, a function neither taking another function as input nor returning a function we will refer to as a _first-order function_)

In R you have encountered these when, for instance, using the `apply` family of functions, which are all versions of what is called a `map` function in functional programming (see below).

When using higher-order functions, it is often useful to create simple anonymous functions at the place in the code where they are used, rather than defining a new named function in one place only to call it in a single other place. In R, all functions are created in this way with the `function` keyword, but they are usually assigned to global names with standard assignment (`<-`). Python provides similar functionality using the `lambda` keyword (name inspired by Alonzo Church's [$\lambda$-calculus](https://www.youtube.com/watch?v=eis11j_iGMs) which has inspired much of functional programming) with which we can create anonymous functions. Of course, we can also pass named functions to higher-order functions, which is usually the case when the function is predefined, general enough to be used in more than one place, or complex enough to warrant separate definition and documentation for the sake of clarity.

## 2.1 The three standard functions `map`, `reduce` and `filter`

There are three standard cases which are widely applicable and many other higher-order functions are special cases or combinations of these. They are: `map`, apply a function on each element in a sequence, `filter`, keep (or conversely, remove) elements from a sequence according to some condition, and `reduce`, combine the elements in a sequence. The `map` function takes a sequence and a function (usually of 1 parameter) which is to be applied to each element of the sequence and might return anything, this function is assumed not to have side effects. The `filter` function takes a function (usually of 1 parameter) which returns a boolean value used to indicate which elements are to be kept. The `reduce` function takes a function (usually of 2 parameters) which is used to combine the elements in the sequence.

In Python, `map` and `filter` are standard built-in functions. Since Python 3, the `reduce` function needs to be imported from the `functools` module.

Many more advanced functions, of any order, can be created by combining these three higher-order functions.

A note from last year: usually, the `reduce` function is more difficult to grasp than `map` and `filter` but I found this blog-post by André Burgaud to be a nice introduction to `reduce`. Note that Burgaud talks about the more general _fold_ concept rather than `reduce`, which is a special case of fold often called _left fold_ (this is covered in more detail in the post). https://www.burgaud.com/foldl-foldr-python/

a) Implement a function `mysum` which computes the sum of a list or tuple of numbers using the reduce function and a lambda function.

In [16]:
from functools import reduce

def mysum(seq):
    return reduce(lambda x, y: x + y, seq)

mysum((4, 7, 1))

12

b) Implement a function `mylength` which uses `map` and `reduce` (or only `reduce`, if you feel like it) to compute the length of a sequence. The use of the `len` function is not allowed.

[Hint: You can use `map` to convert the input to something which can easily be `reduce`:d.]

In [17]:
def mylength(seq):
    return reduce(lambda x, y: x + y, map(lambda x: 1, seq))

mylength((10, 22, 53, 74, 58))

5

## 2.2 Returning functions

The previous section covered functions which take other functions as input, but what about the opposite, functions returning functions as output?

a) Function composition is a common in both maths and programming. Write a function `compose` which takes two functions, $f$ and $g$, and produces the _composite_ function $f \circ g$, where $(f \circ g)(x) \Leftrightarrow f(g(x))$. Example use is given below.

In [18]:
from statistics import stdev, mean

def compose(f, g):
    return lambda x: f(g(x))

def myscale(vals):
    return [x/stdev(vals) for x in vals]

def myshift(vals):
    return [x-mean(vals) for x in vals]

standardize = compose(myscale, myshift)

print(standardize(range(-3, 8)))


[-1.507556722888818, -1.2060453783110545, -0.9045340337332909, -0.6030226891555273, -0.30151134457776363, 0.0, 0.30151134457776363, 0.6030226891555273, 0.9045340337332909, 1.2060453783110545, 1.507556722888818]


**Voluntary task (not required)**
Create a function `composition(*funs)` which takes a non-empty sequence of functions of one argument and returns their sequential composition. That is $composition(f_0,f_1, \ldots, f_n) = f_0 \circ f_1 \circ\ldots \circ f_n$. (The question of if $f\circ g \circ h$ should be read $f\circ (g\circ h)$ or $(f \circ g) \circ h$ is perfectly valid, but they turn out to be the same. That is, $\circ$ is associative.)

In [19]:
# FIXME: this trash don't work
def composition(*funs):
    return reduce(compose, funs)

def myplusone(vals):
    return [x * 10 for x in vals]

fun = composition(myscale, myshift, myplusone)
print(fun(range(-5, 5)))

[-1.486301082920587, -1.1560119533826787, -0.8257228238447705, -0.4954336943068623, -0.1651445647689541, 0.1651445647689541, 0.4954336943068623, 0.8257228238447705, 1.1560119533826787, 1.486301082920587]


Hint: Don't remember what can be found in `*funs`? Print it! Don't know how the values should be combined? Write out some simple example on paper.

Note: This task demonstrates the generality of our constructs. Previously we worked with sequences of numbers and the like. Now we lift this to the level of working with functions as values, and instead of using combinators which work on numbers, we use function combinators in conjunction with our known patterns.

### Voluntary task: pipelining

When doing data analysis, one very important part is pre-processing. Often, data goes through a number of steps of preprocessing, sometimes called a pipeline. The function composition example above can be seen as a special case of such a pipeline for only two functions. By clever use of higher order functions, we can build a pipeline function which takes a list or tuple of data transforming functions and creates a function which applies these sequentially. Construct such a function called `make_pipeline`. In order to focus on the primary purpose of the `make_pipeline` function, we will perform a very simple set of transformations, increment each value by 1, take the absolute value, and then take the square root. Usage example and code for the `inc` function is supplied below.

You may want to use functions you have defined above.

In [20]:
from functools import reduce, partial
from math import sqrt 


# def make_pipeline(*funs):
#     return lambda vals: ???

# We can even drop the lambda vals : bit, using partial
# evaluation (see the help for functools.partial!)

def inc(x):
    return x+1

# pipeline = make_pipeline(inc, abs, sqrt)

# tuple(pipeline(range(-5,5)))

## 3. Simple declarative Pythonic patterns (involving higher order functions)

a) As preparation, create a named tuple type "coord" which has fields `x` and `y`.

In [21]:
from collections import namedtuple
coord = namedtuple("coord", "x y")

five_three = coord(5,3)
assert five_three.x == 5, "first element is the x coordinate"
assert five_three.y == 3, "the second element is the y coordinate"

b) Generate a $10^7$ random coordinates, with $x$ and $y$ coordinates drawn uniformly from [-1000,1000]. Save the tuple of those with $x + y > 0$ as `rnd_coords`. How many are there (probably fewer than $10^7$)?

In [22]:
from random import uniform

rnd_coords = tuple([tup_coord for tup_coord in [coord(uniform(-1000, 1000), uniform(-1000, 1000)) for x in range(int(10e7))] if tup_coord.x + tup_coord.y > 0])
len(rnd_coords)

49997723

[Note: If this takes a while, you might want to consider when the elements are generated and saved!]

**Before having solved the tasks below, consider setting `coords` to a smaller set (eg generate $10^3$ elements instead of $10^7$ to start with).**

c) Let `sorted_rnd` be the coordinated sorted first by the `x` component and then the `y`. Use a built-in Python sorting function. Do you need any extra parameters? Why? Why not? How would you find out where the order comes from (and might it be consistent but useless, eg sorting the elements by memory location)?

In [43]:
rnd_coords = tuple([tup_coord for tup_coord in [coord(uniform(-1000, 1000), uniform(-1000, 1000)) for x in range(int(10e3))] if tup_coord.x + tup_coord.y > 0])
sorted_rnd = sorted(rnd_coords)

# Sorting by first element in each sub-sequence is the default behavior
sorted_rnd[:10]

[coord(x=-969.8752966012545, y=975.1661962247017),
 coord(x=-951.900114068927, y=998.2764898337384),
 coord(x=-942.4785273299136, y=988.8821747336897),
 coord(x=-939.4360679483116, y=948.6211783860058),
 coord(x=-935.1501883085614, y=960.6821796696752),
 coord(x=-930.3447310626651, y=992.5372025113411),
 coord(x=-903.6951479107012, y=912.3614637678768),
 coord(x=-898.2430857442519, y=994.4307144742149),
 coord(x=-896.2504547667571, y=968.8350523822471),
 coord(x=-891.809409264791, y=996.9209088085845)]

[General words of advice:

* During testing, you might want to use a smaller data set (and then try it out at a larger set).
* You might not want to display the entire list to see if you're right all the time. Slicing out the first and last elements, say the first or last 10, might provide some hints.
* You could naturally define a function which checks that the list is in order (or performs some probabilistic sampling test), to test this.]

d) Sort the values (in the sense of returning a new sorted tuple) by their Euclidean distance to the point (5,3). Continue using a built-in Python sorting function.

In [45]:
pts_near_53 = sorted(rnd_coords, key=lambda x: sqrt((5 - x.x)**2 + (3 - x.y)**2))

pts_near_53[:10]

[coord(x=22.55016895135259, y=16.6004812565443),
 coord(x=-4.449957031928534, y=26.951716553947335),
 coord(x=26.05139050455955, y=32.73034602358598),
 coord(x=40.86068452375298, y=-14.627914900071232),
 coord(x=45.198736293916454, y=7.142667619702593),
 coord(x=28.64157162533229, y=37.00351832680985),
 coord(x=-18.82039849643877, y=44.7048910822939),
 coord(x=47.99874820370769, y=-22.258944839683522),
 coord(x=58.171158385810486, y=-3.6775807767626247),
 coord(x=26.450085921122763, y=53.79654895813064)]

Note: here we customise the behaviour of a built-in function by passing it information about our intended ordering.

e) Define the function `sorted_by_distance(origo)` which takes a coordinate `origo` and returns a function which sorts the sequence by the euclidean distance to `origo`. (Ie those closest to origo come first in the list.)

In [48]:
def sorted_by_distance(origo):
    return lambda x: sorted(x, key=lambda x: sqrt((origo.x - x.x)**2 + (origo.y - x.y)**2))

ordered_by_closeness_to_53 = sorted_by_distance(coord(5,3))   # Return the function.
pts_near_53_2 = ordered_by_closeness_to_53(rnd_coords)     # Applying the function 

assert pts_near_53 == pts_near_53_2

[Note: Here we extend the work above to a higher-order function, which uses the local value of `origo`. In essence, this task summarises higher order functionality - we create a closure, return a function and use a custom ordering ]

f) So far in the course, we have seen, and possibly used `enumerate`, `range`, `zip`, `map` and `filter` as declarative constructs (along with the general comprehension syntax). Now we introduce a further useful iterator construct. Construct something called `reverse_squared` which when prompted would give us the squares of elements 0,...,N _but in reverse_ (that is $N^2, (N-1)^2, ..., 2^2, 1^2, 0^2$). You may not give `range` a negative step length in this task (we are interested in more general ways of expressing that we want to iterate in another order, which in this and many cases will prove efficient).

In [57]:
# The time it takes to run this shouldn't really depend on if you use SMALL_N or BIG_N.

BIG_N = 99999999
SMALL_N = 999

N = SMALL_N    # change this to test later on

reverse_squares = [x**2 for x in range(N + 1)][::-1]
reverse_squares[-10:]

[81, 64, 49, 36, 25, 16, 9, 4, 1, 0]

In [59]:
# Experimentation: copy and paste your code from above into this cell.
# This is rather crude, but we want you to to be able to trust that any
# slowness in the cell above can be found by reference to that code, not the
# profiling code below.

# Copy-pasting as it might be useful to have fresh maps.

import profile

# We cut and paste this code 
BIG_N = 99999999
SMALL_N = 999

N = BIG_N    
# Look at the run time. Switching from BIG_N to SMALL_N shouldn't really matter.
# This suggests that we have quick access to elements at the end of our (squared) range.

reverse_squares = [x**2 for x in range(N + 1)][::-1]


profile.run("print(f'Did we find it? ', N**2 in reverse_squares)")

Did we find it?  True
         45 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 :0(__exit__)
        1    0.000    0.000    0.000    0.000 :0(acquire)
        1    0.000    0.000    0.000    0.000 :0(append)
        1    0.000    0.000    0.000    0.000 :0(exec)
        4    0.000    0.000    0.000    0.000 :0(getpid)
        4    0.000    0.000    0.000    0.000 :0(isinstance)
        4    0.000    0.000    0.000    0.000 :0(len)
        1    0.000    0.000    0.000    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        4    0.000    0.000    0.000    0.000 :0(write)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 iostream.py:202(schedule)
        4    0.000    0.000    0.000    0.000 iostream.py:429(_is_master_process)
        4    0.000    0.000    0.000

Note: once you know of the construct, this task is extremely simple. It mostly serves as a demonstration of the availability of these constructs, and how they can be combined. Also, it points to efficiency considerations when using declarative iterator constructs as opposed to fixed computed structures.

As the profiling code above suggests, where we redefine the object in every run, we do not have a purely functional construct. In that case, we wouldn't be able to exhaust the values.

[Additional reading: some additional tools are available in the `itertools` module.]

## 4 Mutating function state

A function always has access to the environment in which it was created. Usually, this means that the function can access global variables. It also means that it can access and modify local bindings from where it was created.

A closure is a function which has access to an environment which is not accessible from outside the function (but which is not destroyed when the function returns). I.e. it is a way to introduce a small measure of statefulness into functional programming. In Python, iterators and generators work much like this. However, we can use the general concept in many cases.

a) Implement a function `make_counter` which has a single parameter `n` which acts as the initial value for a counter. The function should return a function with no parameters which, when called, increments the value of `n` by 1 and returns the new value.

In [78]:
# TODO: solve
def make_counter(n):
    yield lambda x: n+1


counter_A = make_counter(0)
counter_B = make_counter(15)
print("To show that the functions do not affect each others' states, consider the printout:")
print(f"counter_A returns: {counter_A()}")
print(f"counter_A returns: {counter_A()}")
print(f"counter_B returns: {counter_B()}")
print(f"counter_A returns: {counter_A()} (was it affected by the call to counter_B?)")

To show that the functions do not affect each others' states, consider the printout:


TypeError: 'generator' object is not callable

## Attribution

Lab by Johan Falkenjack (2018), extended and rewritten by Anders Märak Leffler (2019).

License [CC-BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)